In [3]:
import requests
from bs4 import BeautifulSoup as BS
from user_agent import generate_user_agent, generate_navigator

import pandas as pd

from tqdm.auto import tqdm
import re

c:\Users\rladb\workspace\dev\github\fifamus\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
crawled_df = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/crawleddf.csv')

In [28]:
crawled_df

,id,name,class,pay,ovr,position,age,height,weight,bodytype,...,strength,stamina,aggression,jump,composure,gk_diving,gk_handle,gk_kick,gk_reflex,gk_positioning
0,101000001,데이비드 시먼,ICON,20,101,GK,61.0,193,93,보통,...,82,70,51,77,86,98,105,106,101,103
1,101000051,앨런 시어러,ICON,25,107,ST,54.0,182,78,보통,...,104,105,95,104,113,23,21,25,19,21
2,101000195,로비 파울러,ICON,23,101,ST,49.0,175,72,보통,...,95,95,89,97,101,15,19,16,18,18
3,101000240,로이 킨,ICON,24,105,CDM,53.0,180,76,보통,...,105,108,115,89,100,22,25,17,17,19
4,101000241,라이언 긱스,ICON,24,106,LM,51.0,179,71,마름,...,98,110,107,97,110,23,24,24,26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61986,511257540,A. 크나우프,22PL,13,80,RM,22.0,180,73,마름,...,61,79,63,71,72,22,26,26,21,27
61987,511257889,K. 야키치,22PL,14,80,CDM,27.0,181,75,보통,...,86,90,90,71,76,22,22,18,22,22
61988,511259307,M. 구스토,22PL,12,79,RB,21.0,179,74,마름,...,72,87,80,75,79,23,20,27,24,22
61989,511262621,G. 마마르다슈빌리,22PL,12,81,GK,24.0,199,88,마름,...,79,54,36,61,37,84,83,79,84,85


In [7]:
player_df = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/playerdf.csv')
df_10001 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_10001.csv')
df_10001_20001 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_10001_20001.csv')
df_20001_30001 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_20001_30001.csv')
df_30001_40001 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_30001_40001.csv')
df_40001_50001 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_40001_50001.csv')
df_61990 = pd.read_csv('C:/Users/rladb/workspace/data/fifafow/df_61990.csv')

In [8]:
player_df

,id,name,class
0,101000001,데이비드 시먼,ICON
1,101000051,앨런 시어러,ICON
2,101000195,로비 파울러,ICON
3,101000240,로이 킨,ICON
4,101000241,라이언 긱스,ICON
...,...,...,...
61986,511257540,A. 크나우프,22PL
61987,511257889,K. 야키치,22PL
61988,511259307,M. 구스토,22PL
61989,511262621,G. 마마르다슈빌리,22PL


In [ ]:
def dictsetter():
    temp = {'pay': [], 'ovr': [], 'position': [], 'age': [], 'height': [], 'weight': [], 'bodytype': [], \
            'skill': [], 'leftfoot': [], 'rightfoot': [], 'reputation': [], 'nation': [], 'o_pac': [], \
            'o_sho': [], 'o_pas': [], 'o_dri': [], 'o_def': [], 'o_phy': [], 'sprint': [], 'accel': [], \
            'finish': [], 'shotpower': [], 'longshot': [], 'positioning': [], 'volley': [], 'penalty': [], \
            'shortpass': [], 'vision': [], 'cross': [], 'longpass': [], 'freekick': [], 'curve': [], 'dribble': [], \
            'control': [], 'agility': [], 'balance': [], 'reaction': [], 'marking': [], 'tackle': [], \
            'intercept': [], 'header': [], 'sliding': [], 'strength': [], 'stamina': [], 'aggression': [], \
            'jump': [], 'composure': [], 'gk_diving': [], 'gk_handle': [], 'gk_kick': [], 'gk_reflex': [], \
            'gk_positioning': [], }
    return temp

In [5]:
def crawler (df):
    result = dictsetter()
    for x in tqdm(df['id'].tolist()):
        url = f'https://fifaonline4.nexon.com/DataCenter/PlayerInfo?spid={x}&n1Strong=0'
        ranheaders = {'User-Agent': generate_user_agent(os='win', device_type='desktop')}
        html = requests.get(url, headers=ranheaders).text
        soup = BS(html, 'html.parser')
        body = soup.find('body')

        info_wrap = body.find('div', class_='info_wrap')
        middle = body.find('div', class_='content_middle').find('ul')
        middle_values = middle.findAll('div', class_=re.compile('value +'))
        bottom = body.find('div', class_='content_bottom')
        bottom_wrap = bottom.findAll('ul', class_=re.compile('data_wrap+'))

        result['pay'].append(info_wrap.find('div', class_='pay_side').text.strip())
        result['ovr'].append(body.find('div', class_='ovr value').text.strip())
        result['position'].append(body.find('div', class_='position').text.strip())
        result['age'].append(info_wrap.find('span', class_='etc birth').text.strip()[-4:-2])
        result['height'].append(info_wrap.find('span', class_='etc height').text.strip()[:3])
        result['weight'].append(info_wrap.find('span', class_='etc weight').text.strip()[:2])
        result['bodytype'].append(info_wrap.find('span', class_='etc physical').text.strip())
        result['skill'].append(len(info_wrap.find('span', class_='etc skill').find('span').text.strip()))
        result['leftfoot'].append(info_wrap.find('span', class_='etc foot').text.strip()[1])
        result['rightfoot'].append(info_wrap.find('span', class_='etc foot').text.strip()[-1])
        result['reputation'].append(info_wrap.find('span', class_='etc season').text.strip())
        result['nation'].append(info_wrap.find('div', class_='etc nation').find('span', class_='txt').text.strip())
        result['o_pac'].append(middle_values[0].text.strip())
        result['o_sho'].append(middle_values[1].text.strip())
        result['o_pas'].append(middle_values[2].text.strip())
        result['o_dri'].append(middle_values[3].text.strip())
        result['o_def'].append(middle_values[4].text.strip())
        result['o_phy'].append(middle_values[5].text.strip())
        result['sprint'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[0].text.strip())
        result['accel'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[1].text.strip())
        result['finish'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[2].text.strip())
        result['shotpower'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[3].text.strip())
        result['longshot'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[4].text.strip())
        result['positioning'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[5].text.strip())
        result['volley'].append(bottom_wrap[0].findAll('div', class_=re.compile('value+'))[6].text.strip())
        result['penalty'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[0].text.strip())
        result['shortpass'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[1].text.strip())
        result['vision'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[2].text.strip())
        result['cross'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[3].text.strip())
        result['longpass'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[4].text.strip())
        result['freekick'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[5].text.strip())
        result['curve'].append(bottom_wrap[1].findAll('div', class_=re.compile('value+'))[6].text.strip())
        result['dribble'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[0].text.strip())
        result['control'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[1].text.strip())
        result['agility'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[2].text.strip())
        result['balance'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[3].text.strip())
        result['reaction'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[4].text.strip())
        result['marking'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[5].text.strip())
        result['tackle'].append(bottom_wrap[2].findAll('div', class_=re.compile('value+'))[6].text.strip())
        result['intercept'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[0].text.strip())
        result['header'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[1].text.strip())
        result['sliding'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[2].text.strip())
        result['strength'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[3].text.strip())
        result['stamina'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[4].text.strip())
        result['aggression'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[5].text.strip())
        result['jump'].append(bottom_wrap[3].findAll('div', class_=re.compile('value+'))[6].text.strip())
        result['composure'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[0].text.strip())
        result['gk_diving'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[1].text.strip())
        result['gk_handle'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[2].text.strip())
        result['gk_kick'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[3].text.strip())
        result['gk_reflex'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[4].text.strip())
        result['gk_positioning'].append(bottom_wrap[4].findAll('div', class_=re.compile('value+'))[5].text.strip())

    temp_df = pd.DataFrame(data = result)
    return temp_df

In [15]:
crawled_df = crawler(player_df)
crawled_df

100%|██████████| 61991/61991 [2:53:09<00:00,  5.97it/s]   


,pay,ovr,position,age,height,weight,bodytype,skill,leftfoot,rightfoot,...,strength,stamina,aggression,jump,composure,gk_diving,gk_handle,gk_kick,gk_reflex,gk_positioning
0,20,101,GK,61,193,93,보통,1,3,5,...,82,70,51,77,86,98,105,106,101,103
1,25,107,ST,54,182,78,보통,3,4,5,...,104,105,95,104,113,23,21,25,19,21
2,23,101,ST,49,175,72,보통,4,5,3,...,95,95,89,97,101,15,19,16,18,18
3,24,105,CDM,53,180,76,보통,2,3,5,...,105,108,115,89,100,22,25,17,17,19
4,24,106,LM,51,179,71,마름,4,5,3,...,98,110,107,97,110,23,24,24,26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61986,13,80,RM,22,180,73,마름,3,3,5,...,61,79,63,71,72,22,26,26,21,27
61987,14,80,CDM,27,181,75,보통,2,3,5,...,86,90,90,71,76,22,22,18,22,22
61988,12,79,RB,21,179,74,마름,3,3,5,...,72,87,80,75,79,23,20,27,24,22
61989,12,81,GK,24,199,88,마름,1,5,3,...,79,54,36,61,37,84,83,79,84,85


In [16]:
crawled_df = pd.concat([player_df, crawled_df], axis=1)
crawled_df

,id,name,class,pay,ovr,position,age,height,weight,bodytype,...,strength,stamina,aggression,jump,composure,gk_diving,gk_handle,gk_kick,gk_reflex,gk_positioning
0,101000001,데이비드 시먼,ICON,20,101,GK,61,193,93,보통,...,82,70,51,77,86,98,105,106,101,103
1,101000051,앨런 시어러,ICON,25,107,ST,54,182,78,보통,...,104,105,95,104,113,23,21,25,19,21
2,101000195,로비 파울러,ICON,23,101,ST,49,175,72,보통,...,95,95,89,97,101,15,19,16,18,18
3,101000240,로이 킨,ICON,24,105,CDM,53,180,76,보통,...,105,108,115,89,100,22,25,17,17,19
4,101000241,라이언 긱스,ICON,24,106,LM,51,179,71,마름,...,98,110,107,97,110,23,24,24,26,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61986,511257540,A. 크나우프,22PL,13,80,RM,22,180,73,마름,...,61,79,63,71,72,22,26,26,21,27
61987,511257889,K. 야키치,22PL,14,80,CDM,27,181,75,보통,...,86,90,90,71,76,22,22,18,22,22
61988,511259307,M. 구스토,22PL,12,79,RB,21,179,74,마름,...,72,87,80,75,79,23,20,27,24,22
61989,511262621,G. 마마르다슈빌리,22PL,12,81,GK,24,199,88,마름,...,79,54,36,61,37,84,83,79,84,85


In [ ]:
temp_nation = []

In [ ]:
for i in range(0, 61991):
    x = crawled_df['nation'].iloc[i].replace(', 국가대표', '')
    temp_nation.append(x)

In [ ]:
crawled_df['nation'] = temp_nation

In [ ]:
crawled_df.to_csv('C:/Users/rladb/workspace/data/fifafow/crawleddf.csv', header=True, index=False, encoding='utf-8-sig')

In [112]:
# info_wrap = body.find('div', class_='info_wrap')
# middle = body.find('div', class_='content_middle').find('ul')
# middle_values = middle.findAll('div', class_=re.compile('value +'))
# bottom = body.find('div', class_='content_bottom')
# bottom_wrap = bottom.findAll('ul', class_=re.compile('data_wrap+'))
#pay
# body.find('div', class_='pay').text.strip()
#ovr
# body.find('div', class_='ovr value').text.strip()
#position
# body.find('div', class_='position').text.strip()
#age
# info_wrap.find('span', class_='etc birth').text.strip()[-4:-2]
#height
# info_wrap.find('span', class_='etc height').text.strip()[:3]
#weight
# info_wrap.find('span', class_='etc weight').text.strip()[:2]
#bodytype
# info_wrap.find('span', class_='etc physical').text.strip()
#skill
# len(info_wrap.find('span', class_='etc skill').find('span').text.strip())
#leftfoot
# info_wrap.find('span', class_='etc foot').text.strip()[1]
#rightfoot
# info_wrap.find('span', class_='etc foot').text.strip()[-1]
#reputation
# info_wrap.find('span', class_='etc season').text.strip()
#nation
# info_wrap.find('div', class_='etc nation').find('span', class_='txt').text.strip()
#ovr 6 stats
# middle.findAll('div', class_=re.compile('value +'))[0].text.strip()
#34 stats (first 7)
# bottom_wrap[0].findAll('div', class_=re.compile('value+'))[0].text.strip()